In [2]:
import os
import gc

import pandas as pd
import numpy as np
import torch

from transformers import PatchTSTForPrediction

from torch.utils.data import DataLoader
from datasets import Dataset

In [3]:
data = "coin"

output_dir = "saved_models"
log_dir = os.path.join('logstf', data)

loss_name = "MASE"

num_train_epochs = 300
model_num = 1
model_path = "./saved_models"
learning_rate = 1e-6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
target_y = pd.read_csv(f"../data/{data}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)
target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)

test_X  = pd.read_csv(f"../data/{data}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
test_y  = pd.read_csv(f"../data/{data}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

In [5]:
def array_to_dataset(X, y):
    X, y = torch.tensor(X), torch.tensor(y)
    X = X.reshape(-1, X.shape[1], 1)
    y = y.reshape(-1, y.shape[1], 1)

    dataset = torch.utils.data.TensorDataset(X, y)

    return dataset

train_dataset = array_to_dataset(target_X, target_y)
val_dataset = array_to_dataset(target_X_val, target_y_val)
test_dataset = array_to_dataset(test_X, test_y)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 64)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 64)

In [6]:
class TransferModel(torch.nn.Module):
    """
    Backbone 전용 아키텍쳐.
    backbone_model.head.projection 자리에 삽입
    backbone_model.head.dropout 자리에 torch.nn.Identity() 삽입 (드롭아웃 레이어를 제거하는 방법을 찾는 것이 더 깔끔하지만, 임시로)
    """
    def __init__(self, t_out):
        super().__init__()
        self.t_out = t_out  ## 24. target_y.shape[1]

        self.adapter = torch.nn.Linear(1792, self.t_out * 128)   ## Dense(128)

        self.head = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128, 64),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(64, 1)
        )

    def forward(self, x):
        self.adapted_feat = self.adapter(x)
        head_input = self.adapted_feat.view(-1, self.t_out, 128)  ## (B, 24, 128)
        output = self.head(head_input)

        return output

In [7]:
T_OUT = target_y.shape[1]

In [8]:
for k in range(1, model_num+1):
    current_path = os.path.join(model_path, f"model_{loss_name}_{k}.pth")

    model_instance = PatchTSTForPrediction.from_pretrained(os.path.join(model_path, "PatchTSTBackbone"))
    model_instance.load_state_dict(torch.load(current_path))

    ## MLP 헤드 부착
    model_instance.head.projection = TransferModel(T_OUT)
    model_instance.head.dropout = torch.nn.Identity()
    model_instance.to(device)

In [9]:
## custom loss function
def SMAPE(yhat, y):
    numerator = 100*torch.abs(y - yhat)
    denominator = (torch.abs(y) + torch.abs(yhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

def MAPE(y_pred, y_true, epsilon=1e-7):
    denominator = torch.clamp(torch.abs(y_true), min=epsilon)       ## 분모에 0이 들어오는 것을 방지
    abs_percent_error = torch.abs((y_true - y_pred) / denominator)

    return torch.mean(100. * abs_percent_error)


class MASE(torch.nn.Module):
    def __init__(self, training_data, period = 1):
        super().__init__()
        ## 원본 코드 구현, 사실상 MAE와 동일, 잘못 짜여진 코드, 일단은 하던대로 할 것.
        self.scale = torch.mean(torch.abs(torch.tensor(training_data[period:] - training_data[:-period])))
    
    def forward(self, yhat, y):
        error = torch.abs(y - yhat)
        return torch.mean(error) / self.scale

In [10]:
optimizer = torch.optim.Adam(model_instance.parameters(), lr = learning_rate)
log_data = []

if loss_name == "mse":
    loss_fn = torch.nn.MSELoss()
elif loss_name == "mae":
    loss_fn = torch.nn.L1Loss()
elif loss_name == "SMAPE":
    loss_fn = SMAPE
elif loss_name == "mape":
    loss_fn = MAPE
elif loss_name == "MASE":
    loss_fn = MASE(target_y, target_y.shape[1])
else:
    raise Exception("Your loss name is not valid.")

## early stopping
PATIENCE = 20
best_val_loss = np.inf
patience_counter = 0

for epoc in range(num_train_epochs):
    model_instance.train()

    total_train_loss = 0

    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        yhat = model_instance(X).prediction_outputs
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()*X.shape[0]

    avg_train_loss = total_train_loss/len(train_dataloader.dataset)

    model_instance.eval()

    with torch.no_grad():
        yys = []
        yyhats = []

        for XX, yy in val_dataloader:
            XX = XX.to(device)
            yys.append(yy.to(device))
            yyhats.append(model_instance(XX).prediction_outputs)

        yyhat = torch.concat(yyhats)
        yy = torch.concat(yys)

        val_loss = loss_fn(yyhat, yy).item()

    print(f"Epoch {epoc+1}/{num_train_epochs} | Train Loss: {avg_train_loss:.6f}\t\t Val Loss: {val_loss:.6f}")

    log_data.append({"epoch": epoc, "loss": avg_train_loss, "eval_loss": val_loss})

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state_dict = model_instance.state_dict()   ## 저장 없이 결과물만 산출...
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        break

Epoch 1/300 | Train Loss: 0.360258		 Val Loss: 0.108685
Epoch 2/300 | Train Loss: 0.354039		 Val Loss: 0.107465
Epoch 3/300 | Train Loss: 0.347369		 Val Loss: 0.105266
Epoch 4/300 | Train Loss: 0.338614		 Val Loss: 0.103429
Epoch 5/300 | Train Loss: 0.331023		 Val Loss: 0.100298
Epoch 6/300 | Train Loss: 0.325545		 Val Loss: 0.095825
Epoch 7/300 | Train Loss: 0.311174		 Val Loss: 0.094819
Epoch 8/300 | Train Loss: 0.299749		 Val Loss: 0.091770
Epoch 9/300 | Train Loss: 0.285591		 Val Loss: 0.085413
Epoch 10/300 | Train Loss: 0.276684		 Val Loss: 0.083688
Epoch 11/300 | Train Loss: 0.267485		 Val Loss: 0.082602
Epoch 12/300 | Train Loss: 0.268595		 Val Loss: 0.075233
Epoch 13/300 | Train Loss: 0.252494		 Val Loss: 0.069002
Epoch 14/300 | Train Loss: 0.251141		 Val Loss: 0.066718
Epoch 15/300 | Train Loss: 0.244459		 Val Loss: 0.067014
Epoch 16/300 | Train Loss: 0.241532		 Val Loss: 0.063198
Epoch 17/300 | Train Loss: 0.232976		 Val Loss: 0.062862
Epoch 18/300 | Train Loss: 0.228530		 Va

In [11]:
pd.DataFrame(log_data).to_csv(os.path.join(log_dir, f"transfer_{loss_name}_lr{learning_rate}_run{1}.csv"))

model_instance.load_state_dict(best_state_dict)

<All keys matched successfully>

In [12]:
with torch.no_grad():
    yys = []
    yyhats = []

    for XX, yy in test_dataloader:
        XX = XX.to(device)
        yys.append(yy.to(device))
        yyhats.append(model_instance(XX).prediction_outputs)

    yyhat = torch.concat(yyhats)
    yy = torch.concat(yys)

    test_loss = loss_fn(yyhat, yy)

In [13]:
mseLoss = torch.nn.MSELoss()
maeLoss = torch.nn.L1Loss()

def smape(yy, yyhat):
    numerator = 100*abs(yy - yyhat)
    denominator = (abs(yy) + abs(yyhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

print(f"test RMSE: {torch.sqrt(mseLoss(yyhat, yy))}")
print(f"test MAE: {maeLoss(yyhat, yy)}")
print(f"test SMAPE: {smape(yy, yyhat)}")

test RMSE: 3.721747636795044
test MAE: 1.4623256921768188
test SMAPE: 2.091921806335449


/root/anaconda3/envs/TF/lib/python3.12/site-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([528, 24, 1])) that is different to the input size (torch.Size([528, 1, 24])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/root/anaconda3/envs/TF/lib/python3.12/site-packages/torch/nn/modules/loss.py:132: UserWarning: Using a target size (torch.Size([528, 24, 1])) that is different to the input size (torch.Size([528, 1, 24])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
